> 06 아스트로이드
    - 좌우 키로 방향을 바구고, 상하키로 앞뒤로 이동합니다. 가속도가 붙기 때문에 조작하는데 숙달이 필요합니다. 스페이스 키를 눌러 미사일을 발사하고 운석을 모두 파괴하세요

In [4]:
# Aastroid  thr second

import sys
import pygame
from random import randint
from math import cos,sin,radians
from pygame.locals import Rect,QUIT,K_LEFT,K_RIGHT,KEYDOWN,KEYUP,K_SPACE,K_UP,K_DOWN


pygame.init()
pygame.key.set_repeat(5,5)
SURFACE = pygame.display.set_mode((800, 800))
FPSCLOCK = pygame.time.Clock()
#효과음
bang_sound = pygame.mixer.Sound(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\bang.wav")   # 짧은 wav/mp3 파일
eat_sound = pygame.mixer.Sound(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\eat.wav")   # 짧은 wav/mp3 파일
ufo_fly = pygame.mixer.Sound(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\UFO_FLY.WAV")   # 짧은 wav/mp3 파일
# 실행 hit_sound.play()
class Drawable: #전체 그리기 객체의 부모클래스
    def __init__(self,rect):
        self.rect = rect
        self.step= [0,0]
    def move(self): #그리기대상을 이동한다
        rect = self.rect.center
        xpos = (rect[0] + self.step[0]) % 800   #화면밖으로나가면반대편에서나오게한다
        ypos = (rect[1] + self.step[1]) % 800
        self.rect.center = (xpos,ypos)

class Rock(Drawable): #움직이는 바위 클래스
    def __init__(self,pos,size):
        super().__init__(Rect(0,0,size,size))
        self.rect.center = pos
        self.size = size
        self.image = pygame.image.load("rock.png")
        self.theta = randint(0,360)
        self.power = 128/size #바위크기에따라속도조절  크기가 작을 수록 속도가 빠름
        self.step[0] = cos(radians(self.theta)) * self.power    #x방향속도계산으로 매프레임마다 이동량을 구함
        self.step[1] = sin(radians(self.theta)) * -self.power

    def draw(self):#운석을그린다
        rotated = pygame.transform.rotozoom(self.image,self.theta,self.size/64)
        rect = rotated.get_rect()
        rect.center = self.rect.center
        SURFACE.blit(rotated,rect)

    def tick(self): #운석을회전시킨다#운석을 이동한다
        self.theta +=3
        self.move()

class Shot(Drawable): #총알발사 클래스
    def __init__(self):
        super().__init__(Rect(0,0,6,6))
        self.count = 40 #총알이 얼마나 진행했는지를 나타내는 카운터
        self.power = 10 #총알속도
        self.max_count = 40 #총알이 최대 얼마동안 진행할지를 나타내는 카운터
    def draw(self):
        ##3총알을그린다
        if self.count < self.max_count:
            pygame.draw.rect(SURFACE,(255,255,0),self.rect)

    def tick(self):
        ##3총알을이동시킨다
        self.count += 1
        self.move()

class Ship(Drawable): #우주선 클래스
    def __init__(self):
        super().__init__(Rect(355,370,90,60))
        self.theta = 0  #케릭터 방향
        self.power = 0  #내 캐릭터 속도
        self.accel = 0  # 내케릭터 가속도
        self.explode= False
        self.image = pygame.image.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\2_images\ship.png")
        self.bang= pygame.image.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\2_images\bang.png")

    def draw(self): #내케릭터를 그린다
        rotated = pygame.transform.rotate(self.image,self.theta)
        rect = rotated.get_rect()
        rect.center = self.rect.center
        SURFACE.blit(rotated,rect)
        if self.explode:
            SURFACE.blit(self.bang,rect)
    
    def tick(self): #내케릭터를움직인다
        self.power +=self.accel
        self.power *= 0.94
        self.accel *= 0.94
        self.step[0] = cos(radians(self.theta))* self.power
        self.step[1] = sin(radians(self.theta))  * -self.power
        self.move()



def key_event_handler(keymap,ship):  #키 이벤트를 처리한다
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == KEYDOWN:
            if not event.key in keymap:
                keymap.append(event.key)
        elif event.type == KEYUP:
            keymap.remove(event.key)

    if K_LEFT in keymap:
        ship.theta +=5

    elif K_RIGHT in keymap:
        ship.theta -=5

    elif K_UP in keymap:
        ufo_fly.play()
        ship.accel= min(5,ship.accel + 0.2)     #가속도를 0.2씩증가시키는테 5가 넘지 않토록
    elif K_DOWN in keymap:
        ufo_fly.play()
        ship.accel= max(-5,ship.accel -0.1)     #가속도를 0.1씩 줄이는데 최대 -5가 넘지않토록


def main():
    sysfont= pygame.font.SysFont(None,72)    
    scorefont = pygame.font.SysFont(None,36)
    message_clear = sysfont.render("CLEARED!",True,(255,255,255))
    message_over = sysfont.render("GAME OVER",True,(255,255,255))
    message_rect=message_clear.get_rect()
    message_rect.center = (400,400)

    keymap = [] #눌리고 있는 키의 코드를 보유하는 리스트입니다
    shots = []  #총알 객체를 저장하는 리스트입니다
    rocks = []  #바위운석 객체를 저장하는 리스트입니다
    ship = Ship()   #내케릭터
    game_over = False
    score = 0
    back_x,back_y = 0,0 #배경이미지를 이동하는 양
    back_image = pygame.image.load("bg.png")
    back_image = pygame.transform.scale2x(back_image)

    while len(shots) < 7:   #총알은 7개
        shots.append(Shot())

    while len(rocks) <4:        #운석은 4개를 초기화하고 배열합니다.
        pos = (randint(0,800),randint(0,800))
        rock = Rock(pos,64)     #랜덤으로 참조된 pos위치에 64크기운석을 rock로 받음
        if not rock.rect.colliderect(ship.rect):    #ship과 중첩이 되지 않토록하기위함
            rocks.append(rock)


    while True:
        key_event_handler(keymap,ship)  #키 이벤트를 처리한다
        
        if not game_over:
            ship.tick()
            #운석을이동
            for rock in rocks:
                rock.tick()
                if rock.rect.colliderect(ship.rect):
                    ship.explode = True
                    game_over = True


            #총알을 이동
            fire = False
            for shot in shots:
                if shot.count < shot.max_count:
                    shot.tick()
                    
                    #총알과운석의충돌처리
                    hit = None
                    for rock in rocks:
                        if rock.rect.colliderect(shot.rect):
                            bang_sound.play()
                            hit = rock
                    if hit != None:
                        score +=hit.rect.width * 10
                        shot.count = shot.max_count
                        rocks.remove(hit)
                        if hit.rect.width > 16:
                            rocks.append(Rock(hit.rect.center,hit.rect.width /2))
                            rocks.append(Rock(hit.rect.center,hit.rect.width /2))
                        if len(rocks) == 0:
                            game_over =True
                elif not fire and K_SPACE in keymap:
                    eat_sound.play()

                    shot.count =0
                    shot.rect.center = ship.rect.center
                    shot_x = shot.power  * cos(radians(ship.theta))
                    shot_y = shot.power * -sin(radians(ship.theta))
                    shot.step = (shot_x,shot_y)
                    fire = True
            
        #배경그리기
        back_x= (back_x +ship.step[0] /2)% 1600
        back_y= (back_y +ship.step[1] /2)% 1600
        SURFACE.fill((0,0,0))
        SURFACE.blit(back_image,(-back_x,-back_y),(0,0,3200,3200))

        #각종객체그리기
        ship.draw()
        for shot in shots:
            shot.draw()
        for rock in rocks:
            rock.draw()


        #점수나타내기
        score_str= str(score).zfill(6)
        score_image= scorefont.render(score_str,True,(0,255,0))
        SURFACE.blit(score_image,(700,10))

        #메세지나타내기
        if game_over:
            if len(rocks)==0:
                SURFACE.blit(message_clear,message_rect.topleft)
            else:
                bang_sound.play()

                SURFACE.blit(message_over,message_rect.topleft)

        pygame.display.update()
        FPSCLOCK.tick(20)

if __name__ == "__main__":
    main()

SystemExit: 